In [1]:
# -*- coding: utf-8 -*-

"""Contains function to perform GIMME"""

from __future__ import absolute_import

import six
from optlang.symbolics import Zero

from driven.data_sets import ExpressionProfile


def gimme(model, expression_profile, fraction_of_optimum=0.9, condition=None):
    """
    Gene Inactivity Moderated by Metabolism and Expression (GIMME)[1]

    Parameters
    ----------
    model: cobra.Model
        A constraint-based model to perform GIMME on.
    expression_profile: ExpressionProfile
        An expression profile to integrate in the model.
    fraction_of_optimum: float
        The fraction of the Required Metabolic Functionalities.
    condition: str or int, optional (default None)
        The condition from the expression profile. If None, the first condition is used.

    Returns
    -------
    context-specific model: cobra.Model
    inconsistency score: float

    References
    ----------
    .. [1] Becker, S. a and Palsson, B. O. (2008).
           Context-specific metabolic networks are consistent with experiments.
           PLoS Computational Biology, 4(5), e1000082.
           doi:10.1371/journal.pcbi.1000082
    """

    assert isinstance(model, cobra.Model)
    assert isinstance(expression_profile, ExpressionProfile)
    assert isinstance(fraction_of_optimum, float)
    assert isinstance(condition, (str, int, None))

    with model:
        prob = model.problem
        condition = expression_profile.conditions[0] if condition is None else condition
        reaction_profile = expression_profile.to_reaction_dict(condition, model, not_measured_value, normalization)

        if model.objective_direction == 'max':
            fix_obj_const = prob.Constraint(model.objective.expression,
                                            lb=fraction_of_optimum * solution.objective_value,
                                            name="Required Metabolic Functionalities")
        else:
            fix_obj_const = prob.Constraint(model.objective.expression,
                                            ub=fraction_of_optimum * solution.objective_value,
                                            name="Required Metabolic Functionalities")

        model.add_cons_vars(fix_obj_const)
        
        coefficients = {r: cutoff - exp if cutoff > exp else 0 for r, exp in six.iteritems(reaction_profile)}
        obj_vars = []
        for rxn_id, coefficient in six.iteritems(coefficients):
            rxn = model.reactions.get_by_id(rxn_id)
            if coefficient > 0:
                obj_vars.append(coefficient * rxn.flux_expression)

        model.objective = prob.Objective(Zero, sloppy=True, direction="min")
        model.objective.set_linear_coefficients({v: 1.0 for v in obj_vars})
        
        sol = model.slim_optimize()
        
        return (model, sol)

ImportError: No module named 'driven'